In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| default_exp options

# Options

## Selecting feature types and attributes

You can select which feature type to display. The default feature types are `["CDS", "repeat_region", "ncRNA", "rRNA", "tRNA"]`.

You can inspect the gff file to see what feature types and attributes are available. The `genomenotebook.parse_gff` function can conveniently be used for this purpose. It also accepts gzipped gff files. 

In [ ]:
import genomenotebook as gn
import os
from bokeh.io import output_notebook #|hide_line
import numpy as np #|hide_line
from nbdev.showdoc import * #|hide_line

In [ ]:
output_notebook(hide_banner=True) #|hide_line

data_path = gn.get_example_data_dir()
gff_path = os.path.join(data_path, "MG1655_U00096.gff3")
features = gn.parse_gff(gff_path)
features.head()

,seq_id,source,type,start,end,score,strand,phase,attributes,left,right,middle
0,U00096.3,Genbank,region,1,4641652,.,+,.,"{'ID': 'U00096.3:1..4641652', 'Dbxref': 'taxon...",1,4641652,2320826.5
1,U00096.3,Genbank,gene,190,255,.,+,.,"{'ID': 'gene-b0001', 'Dbxref': 'ASAP:ABE-00000...",190,255,222.5
2,U00096.3,Genbank,CDS,190,255,.,+,0,"{'ID': 'cds-AAC73112.1', 'Parent': 'gene-b0001...",190,255,222.5
3,U00096.3,Genbank,gene,337,2799,.,+,.,"{'ID': 'gene-b0002', 'Dbxref': 'ASAP:ABE-00000...",337,2799,1568.0
4,U00096.3,Genbank,CDS,337,2799,.,+,0,"{'ID': 'cds-AAC73113.1', 'Parent': 'gene-b0002...",337,2799,1568.0


In [ ]:
# Available feature types
set(features.type)

{'CDS',
 'exon',
 'gene',
 'mobile_genetic_element',
 'ncRNA',
 'origin_of_replication',
 'pseudogene',
 'rRNA',
 'recombination_feature',
 'region',
 'repeat_region',
 'sequence_feature',
 'tRNA'}

In [ ]:
# Available attributes
features.columns

Index(['seq_id', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase',
       'attributes', 'left', 'right', 'middle'],
      dtype='object')

In [ ]:
#Choosing the feature types, attributes and feature name to display
g=gn.GenomeBrowser(gff_path, 
                   feature_types = ["tRNA","rRNA"],
                   attributes = ["gene","locus_tag","strand","start","end"], #will be displayed when hovering 
                   init_pos=226000)
g.show()

## Changing colors

### Modifying features by type

The glyph shown for each feature type is defined through a Glyph object.
A custom glyphs dictionnary can be passed to GenomeBrowser to customize the glyphs shown for different features.

In [ ]:
glyphs=gn.get_default_glyphs()
glyphs['CDS']

Glyph object with attributes:
	glyph_type: arrow
	colors: ('purple', 'orange')
	height: 1
	alpha: 0.8
	show_name: True
	name_attr: gene

In [ ]:
glyphs['repeat_region']

Glyph object with attributes:
	glyph_type: box
	colors: ('grey',)
	height: 0.8
	alpha: 1
	show_name: False
	name_attr: gene

Modifying the default glyphs

In [ ]:
glyphs["CDS"].colors= ('blue','green')

g=gn.GenomeBrowser(gff_path, glyphs=glyphs, init_pos=224000, bounds=(220000,230000), search=False)
g.show()

Defining a new Glyph from scractch

In [ ]:
glyphs["rRNA"]=gn.Glyph(glyph_type="box", 
                        colors="red", 
                        height=0.5, 
                        show_name=False)
                        
g=gn.GenomeBrowser(gff_path, glyphs=glyphs, init_pos=224000, bounds=(220000,230000), search=False)
g.show()

### Modifying specific features

You can also access a DataFrame with the characteristics of each feature plotted on the genome browser. These can be modified as you wish to customize your plot. 

In [ ]:
g.patches.head()

,names,xs,ys,xbox_min,color,alpha,pos,attributes
0,metQ,"(220928, 220928, 220213, 220113, 220213)","(0.05, 0.2, 0.2, 0.125, 0.05)",220213,green,0.8,220520.5,"<span style=""color:FireBrick"">CDS</span><br><s..."
1,metI,"(221621, 221621, 221068, 220968, 221068)","(0.05, 0.2, 0.2, 0.125, 0.05)",221068,green,0.8,221294.5,"<span style=""color:FireBrick"">CDS</span><br><s..."
2,metN,"(222645, 222645, 221714, 221614, 221714)","(0.05, 0.2, 0.2, 0.125, 0.05)",221714,green,0.8,222129.5,"<span style=""color:FireBrick"">CDS</span><br><s..."
3,gmhB,"(222833, 222833, 223308, 223408, 223308)","(0.05, 0.2, 0.2, 0.125, 0.05)",222833,blue,0.8,223120.5,"<span style=""color:FireBrick"">CDS</span><br><s..."
4,rrsH,"(223771, 223771, 225312, 225312)","(0.0875, 0.1625, 0.1625, 0.0875)",223771,red,0.8,224541.5,"<span style=""color:FireBrick"">rRNA</span><br><..."


In [ ]:
g=gn.GenomeBrowser(gff_path, init_pos=224000, bounds=(220000,230000), search=False)
g.patches.loc[g.patches.names=="metN","color"]="green"
g.show()

## Customizing labels

### Changing the attribute used as the feature name

You can chose which attribute of the GFF file should be displayed on top of the gene. The feature_name needs to belong to the list of attributes. The default list of attributes is \["locus_tag","gene","product"].

In [ ]:
g=gn.GenomeBrowser(gff_path, 
                   attributes=["protein_id",'gene','product'], 
                   feature_name="protein_id",
                   bounds=(20000,30000),
                   search=False)
g.show()

In [ ]:
g.show()

### Changing the name of specific features

You can also modify the names of specific features directly in the patches DataFrame

In [ ]:
g.patches.head()

,names,xs,ys,xbox_min,color,alpha,pos,attributes
0,insB-1,"(20314, 20314, 19911, 19811, 19911)","(0.05, 0.2, 0.2, 0.125, 0.05)",19911,orange,0.8,20062.5,"<span style=""color:FireBrick"">CDS</span><br><s..."
1,insA-1,"(20508, 20508, 20333, 20233, 20333)","(0.05, 0.2, 0.2, 0.125, 0.05)",20333,orange,0.8,20370.5,"<span style=""color:FireBrick"">CDS</span><br><s..."
2,rpsT,"(21078, 21078, 20915, 20815, 20915)","(0.05, 0.2, 0.2, 0.125, 0.05)",20915,orange,0.8,20946.5,"<span style=""color:FireBrick"">CDS</span><br><s..."
3,yaaY,"(21181, 21181, 21299, 21399, 21299)","(0.05, 0.2, 0.2, 0.125, 0.05)",21181,purple,0.8,21290.0,"<span style=""color:FireBrick"">CDS</span><br><s..."
4,,"(21407, 21407, 22248, 22348, 22248)","(0.05, 0.2, 0.2, 0.125, 0.05)",21407,green,0.8,21877.5,"<span style=""color:FireBrick"">CDS</span><br><s..."


In [ ]:
g.patches.loc[g.patches.names=="ribF","color"]="green"
g.patches.loc[g.patches.names=="","names"]="custom"
#g.patches.loc[0,"attributes"]
g.elements

[figure(id='p11167', ...)]

### Changing label angle and size

In [ ]:
g=gn.GenomeBrowser(gff_path,  
                   label_angle=0,
                   label_font_size="10pt",
                   height=80,
                   feature_height=0.4, #fraction of the annotation track occupied by the features
                   bounds=(20000,30000),
                   search=False)
g.show()

### Adding information to be displayed when hovering

Any new column added to the patches DataFrame can be shown when hovering over features of the annotation track. The name of the column to display should also be provided as an attribute.

In [ ]:
g=gn.GenomeBrowser(gff_path, 
                   init_pos=224000, 
                   bounds=(220000,230000), 
                   attributes=["gene","locus_tag","data"], 
                   search=False)
g.patches["data"]=np.random.randint(0,10,len(g.patches))
g.show()

## Toolbar location

The location of the toolbar can be changed for each track independently. Placing it above or below can help see all the Bokeh tools available. Valid values are:
* "above"
* "below"
* "left"
* "right"

See the [Bokeh documentation](https://docs.bokeh.org/en/latest/docs/user_guide/interaction/tools.html#positioning-the-toolbar) for more details.


In [ ]:
import pandas as pd

data_path = gn.get_example_data_dir()
gff_path = os.path.join(data_path, "MG1655_U00096.gff3")

data=pd.DataFrame(dict(x=np.arange(0,50000,100),
                       y=np.sin(np.arange(0,50000,100))))

g=gn.GenomeBrowser(gff_path=gff_path, 
                bounds=(0,50000),
                toolbar_location="above",
                search=False)

track = g.add_track(height=200, 
                    toolbar_location="above")

track.bar(data=data, pos="x", y="y")
g.show()